# Credentials for Snowflake

In [1]:
#Login credentials for sknowflake.
#Ideal scenario these are stored as env variables
User_Name='MITNALAS'
password='@Inferno112'
database='SPOTIFY_KRISHNA_DATA'
schema='SPOTIFY_SCHEMA'
account_name='nvb90940.us-east-1'

# Importing Libraries

In [2]:
#importing libraries
import pandas as pd
import snowflake.connector

In [8]:
#Reading the files that are to be loaded onto the Snowflake databse
Album_rank=pd.read_excel('Album_genre_rank.xlsx')
Album_release=pd.read_excel('album_release.xlsx')
Music_Genre=pd.read_excel('Music_genre.xlsx')
Krishna_music=pd.read_excel('Krishna_music.xlsx')
Song_popularity=pd.read_excel('Song_popularity.xlsx')

# Connecing to snowflake

In [4]:
#Connecting to snowflake
conn = snowflake.connector.connect(
    account=account_name,
    user=User_Name,
    password=password,
    database=database,
    schema=schema
)

In [5]:
#Loaded data from csvs onto the snowflake database
cursor=conn.cursor()

# Creating Tables

In [6]:
#creating tables that are to be added onto the Database
create_krishna_music="""
CREATE TABLE Krishna_Music (
Song_ID varchar(100),
Song_Name varchar(100),
Artist_Name varchar(100),
Album_Name varchar(100),
Played_Date TIMESTAMP,
PRIMARY KEY(Album_Name))
"""

create_song_popular="""
CREATE TABLE Song_popularity (
Artist_name varchar(200),
Track_name varchar(500),
Popularity float(5))
"""

create_album_rank="""
CREATE TABLE Album_rank(
Album_name varchar(100),
Artist_name varchar(100),
Rank int,
Genre varchar(10),
FOREIGN KEY(Album_name) REFERENCES Krishna_music(Album_Name)
)
"""

create_album_release="""
CREATE TABLE Album_release(
Album_name varchar(100),
Album_ID varchar(100),
Number_of_songs int,
Release_Date Date,
PRIMARY KEY(Album_ID),
FOREIGN KEY(Album_name) REFERENCES Krishna_music(Album_Name)
)
"""

create_music_genre="""
CREATE TABLE Music_Genre(
Track_name varchar(500),
acousticness float(5),
danceability float(5),
duration int,
energy float(5),
instrumentalness float(5),
Key varchar(5),
liveness float(10),
loudness float(10),
mode varchar(20),
speechiness float(10),
tempo float(10),
valence float(10),
Genre varchar(20)
)
"""

#cursor.execute(create_krishna_music)
#cursor.execute(create_album_rank)
#cursor.execute(create_album_release)
#cursor.execute(create_music_genre)
cursor.execute(create_song_popular)
conn.commit()

# Transforming Data in the required format

In [7]:
#Remove duplicates from Krishna music
Krishna_music.drop_duplicates(subset='Album Name', keep='first', inplace=True)
len(Krishna_music)

24

In [8]:
#Rename Columns
Krishna_music.rename(columns={'Song ID':'Song_ID','Song Name':'Song_Name','Artist Name':'Artist_Name','Album Name':'Album_Name','Played Date':'Played_Date'},inplace=True)
Krishna_music.head()

,Song_ID,Song_Name,Artist_Name,Album_Name,Played_Date
0,6AQbmUe0Qwf5PZnt4HmTXv,Boy's a liar Pt. 2,PinkPantheress,Boy's a liar Pt. 2,2023-05-14T15:53:57.265Z
1,5fVZC9GiM4e8vu99W0Xf6J,How to Save a Life,The Fray,How To Save A Life,2023-05-14T05:53:47.150Z
2,5hnyJvgoWiQUYZttV4wXy6,Chasing Cars,Snow Patrol,Eyes Open,2023-05-14T05:51:12.437Z
3,4RCWB3V8V0dignt99LZ8vH,Hey There Delilah,Plain White T's,All That We Needed,2023-05-14T05:46:04.159Z
4,2lFlveK1y13WWp3vnQtrr3,100 Years,Five For Fighting,The Battle for Everything,2023-05-14T05:42:11.233Z


In [61]:
#Album_release.rename(columns={'Album Name':'Album_name','Album ID':'Album_ID','Number of Songs':'Number_of_songs','Release Date':'Release_Date'},inplace=True)
Album_release=Album_release.iloc[1:]
Album_release['Album_name']=Album_release['Album_name'].str.strip()
Album_release.dropna(inplace=True)
Album_release.head()

,Album_name,Album_ID,Number_of_songs,Release_Date
4,T R A P S O U L,6eZdwrhB97A3EYx9QppGfl,14,2015-10-02
5,Traveller,7lxHnls3yQNl8B9bILmHj7,14,2015-05-04
6,To Pimp A Butterfly,7ycBtnsMtyVbbwTfJwRjSP,16,2015-03-16
7,25,3AvPX1B1HiFROvYjLb5Qwi,11,2015-11-20
8,Blurryface,3cQO7jp5S9qLBoIVtbkSM1,14,2015-05-15


In [71]:
Album_rank.rename(columns={'Album Name':'Album_name','Artist Name':'Artist_name'},inplace=True)
Album_rank.head()

,Album_name,Artist_name,Rank,Genre
0,Minutes to Midnight,Linkin Park,1,rock
1,Abbey Road (Remastered),The Beatles,2,rock
2,The Beatles (Remastered),The Beatles,3,rock
3,Rumours (Super Deluxe),Fleetwood Mac,4,rock
4,O My Heart,Mother Mother,5,rock


In [92]:
#Music_Genre.rename(columns={'music_genre':'Genre','duration_ms':'duration'},inplace=True)
#Music_Genre.dropna(inplace=True)
Music_Genre=Music_Genre[~Music_Genre[Music_Genre.columns].isin(['?']).any(axis=1)]
Music_Genre.head()

,track_name,acousticness,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,Genre
0,Röyksopp's Night Out,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,0.759,Electronic
1,The Shining Path,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,0.531,Electronic
2,Hurricane,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,0.333,Electronic
3,Nitro,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,0.270,Electronic
4,Divide & Conquer,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,0.323,Electronic


In [14]:
#Song_popularity.drop(columns={'Unnamed: 0'},inplace=True)
Song_popularity.rename(columns={'artist_name':'Artist_name','track_name':'Track_name','popularity':'Popularity'},inplace=True)
Song_popularity.dropna(inplace=True)
Song_popularity.head()

,Artist_name,Track_name,Popularity
0,Röyksopp,Röyksopp's Night Out,27.0
1,Thievery Corporation,The Shining Path,31.0
2,Dillon Francis,Hurricane,28.0
3,Dubloadz,Nitro,34.0
4,What So Not,Divide & Conquer,32.0


# Inserting data into the tables

In [41]:
# Generate placeholders for the values
placeholders = ' , '.join(['%s'] * len(Krishna_music.columns))
columns = ', '.join(Krishna_music.columns)    
# Create the SQL statement for inserting values into the table
insert_query = f"INSERT INTO Krishna_music ({columns}) VALUES ({placeholders})"
cursor.executemany(insert_query,Krishna_music.values.tolist())
print("Table 1 inserted")
conn.commit()


Table 1 inserted


In [62]:
placeholders = ' , '.join(['%s'] * len(Album_release.columns))
columns = ', '.join(Album_release.columns)    
# Create the SQL statement for inserting values into the table
insert_query = f"INSERT INTO Album_release ({columns}) VALUES ({placeholders})"
cursor.executemany(insert_query,Album_release.values.tolist())
print("Table 2 inserted")
conn.commit()


Table 2 inserted


In [74]:
placeholders = ' , '.join(['%s'] * len(Album_rank.columns))
columns = ', '.join(Album_rank.columns)    
# Create the SQL statement for inserting values into the table
insert_query = f"INSERT INTO Album_rank ({columns}) VALUES ({placeholders})"
cursor.executemany(insert_query,Album_rank.values.tolist())
print("Table 3 inserted")
conn.commit()

Table 3 inserted


In [12]:
Music_Genre=Music_Genre.head(16000)
Song_popularity=Song_popularity.head(16000)

In [95]:
placeholders = ' , '.join(['%s'] * len(Music_Genre.columns))
columns = ', '.join(Music_Genre.columns)    
# Create the SQL statement for inserting values into the table
insert_query = f"INSERT INTO Music_Genre ({columns}) VALUES ({placeholders})"
cursor.executemany(insert_query,Music_Genre.values.tolist())
print("Table 4 inserted")
conn.commit()


Table 4 inserted


In [15]:
placeholders = ' , '.join(['%s'] * len(Song_popularity.columns))
columns = ', '.join(Song_popularity.columns)    
# Create the SQL statement for inserting values into the table
insert_query = f"INSERT INTO Song_popularity ({columns}) VALUES ({placeholders})"
cursor.executemany(insert_query,Song_popularity.values.tolist())
print("Table 5 inserted")
conn.commit()

Table 5 inserted
